In [1]:
import findspark 
findspark.init('C:\Spark\spark-3.0.0-preview2-bin-hadoop2.7')

In [2]:
from pyspark import SparkContext
try: 
    type(sc)
except NameError:
    sc = SparkContext('local[*]')

In [3]:
sales = [
    (1, 1, '01', '2017', 500), # sera la minima
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 1, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '02', '2017', 500),
    (4, 3, '03', '2017', 500),    

]

stores = [
    (1 , 'address 1', -1, -1, '30002', 'Georgia'),
    (2 , 'address 2', -2, -2, '30003', 'Georgia'),
    (3 , 'address 2', -3, -3, '30004', 'Georgia'),
    (4 , 'address 2', -4, -4, '10119', 'New York')    
]

In [17]:
# Obtener la tienda que realizó menor cantidad de ventas en el estado de "Georgia" en todo el año 2017

In [4]:
salesRdd = sc.parallelize(sales)
storesRdd = sc.parallelize(stores)

In [9]:
# filtramos la informacion de 2017 de ventas y realizamos un cambio de clave
salesToJoin = salesRdd.filter(lambda x: x[3] == '2017')\
    .map(lambda x: (x[1], (x[1],x[4]))) # (K,V) K: id_tienda, V: (id_tienda, total_ventas_mensuales)

# consideramos solo tiendas de Georgia
storesToJoin = storesRdd.filter(lambda x: x[5] == 'Georgia')\
    .map(lambda x: (x[0], x[0]))

In [10]:
# realizamos el join de ambos data frames
# simplificamos el formato para evitar los reduce complejos
# sumarizamos cantidad de ventas por tienda
# buscamos el minimo con reduce
salesToJoin.join(storesToJoin)\
    .map(lambda x: (x[0], x[1][0][1]))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if x[1] < y[1] else y)

(1, 500)

In [15]:
## PARCIAL 2019 1C - 1°

In [5]:
import pandas as pd
import random
import numpy as np

In [6]:
#Datos aleatorios extraidos de la web para aportar un set ejemplo y utilización de random

provincias = ['Álava', 'Albacete','Alicante','Almería', 'Asturias','Ávila','Badajoz','Barcelona','Burgos','Cáceres','Cádiz','Cantabria','Castellón','Ciudad Real','Córdoba','La Coruña','Cuenca','Gerona','Granada','Guadalajara','Guipúzcoa','Huelva','Huesca','Islas Baleares','Jaén','León','Lérida','Lugo','Madrid','Málaga','Murcia','Navarra','Orense','Palencia','Las Palmas','Pontevedra','La Rioja','Salamanca','Segovia','Sevilla','Soria','Tarragona','Santa Cruz de Tenerife','Teruel','Toledo','Valencia','Valladolid','Vizcaya','Zamora','Zaragoza']

ciudades = ["Andalucía", "Aragón", "Canarias", "Cantabria", "Castilla y León", "Castilla-La Mancha", "Cataluña", "Ceuta", "Comunidad Valenciana", "Comunidad de Madrid", "Extremadura", "Galicia", "Islas Baleares", "La Rioja", "Melilla", "Navarra", "País Vasco", "Principado de Asturias", "Región de Murcia"]

nombres_bases = ["one", "two", "three", "four", "five","six", "seven", "eight", "nine", "ten", "eleven"]

direcciones = ["S","E","O","N","SO","SE","NE","NO","h","m","y","x"]

In [7]:
df = pd.DataFrame({'ID_BASE': random.sample(range(0,11),k=10),
                   'NOMBRE': random.sample(nombres_bases,k=10),
                   'PCIA': random.sample(provincias,k=10),
                   'CIUDAD':random.sample(ciudades,k=10),}
                  )
df

,ID_BASE,NOMBRE,PCIA,CIUDAD
0,7,one,Ávila,La Rioja
1,10,nine,Vizcaya,Región de Murcia
2,9,seven,Las Palmas,Principado de Asturias
3,4,ten,Pontevedra,Extremadura
4,1,two,Toledo,Melilla
5,2,six,Soria,Cataluña
6,0,three,Tarragona,Castilla y León
7,3,four,Lérida,Castilla-La Mancha
8,6,eight,Orense,Islas Baleares
9,5,eleven,Jaén,Comunidad de Madrid


In [8]:

df2 = pd.DataFrame({'TIMESTAMP': random.sample(range(0,20),k=10),
                   'ID_BASE': random.sample(range(0,11),k=10),
                   'TEMPERATURA': random.sample(range(0,40),k=10),
                   'HUMEDAD':random.sample(range(0,100),k=10),
                   'PRESION':random.sample(range(0,100),k=10),
                   'DIRECCION VIENTO':random.sample(direcciones,k=10),
                   'VELOCIDAD VIENTO':random.sample(range(0,100),k=10),}
                  )
df2

,TIMESTAMP,ID_BASE,TEMPERATURA,HUMEDAD,PRESION,DIRECCION VIENTO,VELOCIDAD VIENTO
0,2,10,28,46,3,N,79
1,13,9,33,48,42,x,43
2,17,8,32,79,43,S,41
3,1,5,13,28,70,m,18
4,7,3,2,6,62,NO,69
5,11,0,22,75,96,E,23
6,4,7,8,19,8,O,44
7,15,2,19,14,11,SO,53
8,5,6,24,61,86,NE,6
9,8,4,7,88,99,h,68


In [30]:
rdm = sc.parallelize(df)
rdm2 = sc.parallelize(df2)

In [31]:
rdm.collect()

['ID_BASE', 'NOMBRE', 'PCIA', 'CIUDAD']

In [40]:
rdm2.collect()

['TIMESTAMP',
 'ID_BASE',
 'TEMPERATURA',
 'HUMEDAD',
 'PRESION',
 'DIRECCION VIENTO',
 'VELOCIDAD VIENTO']

In [28]:
values = rdm.map(lambda x: (x[0],x))

In [29]:
values.collect()

[('I', 'ID_BASE'), ('N', 'NOMBRE'), ('P', 'PCIA'), ('C', 'CIUDAD')]

In [19]:
values.take(2)

[('I', 'ID_BASE'), ('N', 'NOMBRE')]

In [23]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
spDF = sqlContext.createDataFrame(df)

In [24]:
spDF.collect()

[Row(ID_BASE=7, NOMBRE='one', PCIA='Ávila', CIUDAD='La Rioja'),
 Row(ID_BASE=10, NOMBRE='nine', PCIA='Vizcaya', CIUDAD='Región de Murcia'),
 Row(ID_BASE=9, NOMBRE='seven', PCIA='Las Palmas', CIUDAD='Principado de Asturias'),
 Row(ID_BASE=4, NOMBRE='ten', PCIA='Pontevedra', CIUDAD='Extremadura'),
 Row(ID_BASE=1, NOMBRE='two', PCIA='Toledo', CIUDAD='Melilla'),
 Row(ID_BASE=2, NOMBRE='six', PCIA='Soria', CIUDAD='Cataluña'),
 Row(ID_BASE=0, NOMBRE='three', PCIA='Tarragona', CIUDAD='Castilla y León'),
 Row(ID_BASE=3, NOMBRE='four', PCIA='Lérida', CIUDAD='Castilla-La Mancha'),
 Row(ID_BASE=6, NOMBRE='eight', PCIA='Orense', CIUDAD='Islas Baleares'),
 Row(ID_BASE=5, NOMBRE='eleven', PCIA='Jaén', CIUDAD='Comunidad de Madrid')]

In [33]:
spDF

DataFrame[ID_BASE: bigint, NOMBRE: string, PCIA: string, CIUDAD: string]

In [34]:
spDF.show()

+-------+------+----------+--------------------+
|ID_BASE|NOMBRE|      PCIA|              CIUDAD|
+-------+------+----------+--------------------+
|      7|   one|     Ávila|            La Rioja|
|     10|  nine|   Vizcaya|    Región de Murcia|
|      9| seven|Las Palmas|Principado de Ast...|
|      4|   ten|Pontevedra|         Extremadura|
|      1|   two|    Toledo|             Melilla|
|      2|   six|     Soria|            Cataluña|
|      0| three| Tarragona|     Castilla y León|
|      3|  four|    Lérida|  Castilla-La Mancha|
|      6| eight|    Orense|      Islas Baleares|
|      5|eleven|      Jaén| Comunidad de Madrid|
+-------+------+----------+--------------------+



In [35]:
rdd = spDF.rdd

In [38]:
values = rdd.map(lambda x: (x[0],x))

In [39]:
values.collect()

[(7, Row(ID_BASE=7, NOMBRE='one', PCIA='Ávila', CIUDAD='La Rioja')),
 (10,
  Row(ID_BASE=10, NOMBRE='nine', PCIA='Vizcaya', CIUDAD='Región de Murcia')),
 (9,
  Row(ID_BASE=9, NOMBRE='seven', PCIA='Las Palmas', CIUDAD='Principado de Asturias')),
 (4, Row(ID_BASE=4, NOMBRE='ten', PCIA='Pontevedra', CIUDAD='Extremadura')),
 (1, Row(ID_BASE=1, NOMBRE='two', PCIA='Toledo', CIUDAD='Melilla')),
 (2, Row(ID_BASE=2, NOMBRE='six', PCIA='Soria', CIUDAD='Cataluña')),
 (0,
  Row(ID_BASE=0, NOMBRE='three', PCIA='Tarragona', CIUDAD='Castilla y León')),
 (3,
  Row(ID_BASE=3, NOMBRE='four', PCIA='Lérida', CIUDAD='Castilla-La Mancha')),
 (6, Row(ID_BASE=6, NOMBRE='eight', PCIA='Orense', CIUDAD='Islas Baleares')),
 (5,
  Row(ID_BASE=5, NOMBRE='eleven', PCIA='Jaén', CIUDAD='Comunidad de Madrid'))]